# 라이브러리 선언

In [1]:
import os
import re
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

# 데이터 전처리

In [2]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names = ['eng', 'fra', 'cc'], sep = '\t')
lines.sample(5)

,eng,fra,cc
161239,You're too suspicious about everything.,Vous êtes trop suspicieuses de tout.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
114040,What do you know about pandas?,Que savez-vous des pandas ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
119422,Thank you for your explanation.,Merci pour ton explication.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
12768,You are strong.,Tu es forte.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
175169,He received quite a few letters this morning.,"Il a reçu pas mal de lettres, ce matin.",CC-BY 2.0 (France) Attribution: tatoeba.org #1...


In [3]:
lines = lines [['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
4150,We like Tom.,Nous aimons bien Tom.
14639,I'd like a beer.,J'aimerais une bière.
12411,Try to do that.,Essaie de faire cela.
14953,Is it not black?,N'est-ce pas noir ?
26615,Tom got kidnapped.,Tom a été kidnappé.


문장 토큰화 함수

In [4]:
sos_token = '<start>'
eos_token = '<end>'

def preprocess(lines, plus_token = True):
    lines = lines.lower().strip()
    
    lines = re.sub(r"([?.!,¿])", r" \1 ", lines)
    lines = re.sub(r'[" "]+', " ", lines)
    lines = re.sub(r"[^a-zA-Z?.!,¿]+", " ", lines)

    lines = lines.strip()
    if plus_token == True:
        lines = '<start> ' + lines + ' <end>'    
    lines = lines.split(' ')
    return lines

In [5]:
lines.eng = lines.eng.apply(lambda x: preprocess(x, plus_token = False))
lines.sample(5)

,eng,fra
9135,"[we, must, speak, .]",Il nous faut parler.
14115,"[i, hate, my, voice, .]",Je déteste ma voix.
11423,"[let, s, sit, down, .]",Asseyons-nous.
2620,"[we, re, busy, .]",Nous sommes occupées.
9118,"[we, have, plans, .]",Nous avons des plans.


In [6]:
lines.fra = lines.fra.apply(lambda x: preprocess(x))
lines.sample(5)

,eng,fra
13973,"[i, can, t, give, up, .]","[<start>, je, n, arrive, pas, arr, ter, ., <end>]"
18886,"[i, love, libraries, .]","[<start>, j, adore, les, biblioth, ques, ., <e..."
270,"[go, away, !]","[<start>, bouge, !, <end>]"
17293,"[you, re, the, best, .]","[<start>, tu, es, le, meilleur, ., <end>]"
4901,"[i, feel, awful, .]","[<start>, je, me, sens, affreusement, mal, ., ..."


In [7]:
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines.eng)

In [8]:
fra_tokenizer = Tokenizer()
fra_tokenizer.fit_on_texts(lines.fra)

In [9]:
input_text = eng_tokenizer.texts_to_sequences(lines.eng)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)

In [10]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1

In [11]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])

In [12]:
print('전체 샘플의 수: ', len(lines))
print('영어 단어장의 크기: ', eng_vocab_size)
print('프랑스어 단어장의 크기: ', fra_vocab_size)
print('영어 시퀸스의 최대 길이: ', max_eng_seq_len)
print('프랑스어 시퀸스의 최대 길이: ', max_fra_seq_len)

전체 샘플의 수:  33000
영어 단어장의 크기:  4672
프랑스어 단어장의 크기:  7455
영어 시퀸스의 최대 길이:  8
프랑스어 시퀸스의 최대 길이:  17


train data 와 test data 준비

In [13]:
encoder_input = input_text

decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [14]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding = 'post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding = 'post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding = 'post')

In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [16]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 8)
(30000, 17)
(30000, 17)
(3000, 8)
(3000, 17)
(3000, 17)


Model 설계 및 훈련

In [17]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, 256, input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units = 256, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [18]:
decoder_inputs = Input(shape=(None,))
dec_emb = Embedding(fra_vocab_size, 256)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
decoder_outputs, _, _= decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [19]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    1196032     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 256)    1908480     input_2[0][0]                    
______________________________________________________________________________________________

In [20]:
model.compile(optimizer="rmsprop", loss='sparse_categorical_crossentropy')

In [21]:
model.fit([encoder_input_train, decoder_input_train],decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 32, epochs = 50)

Epoch 1/50
938/938 [==============================] - 23s 16ms/step - loss: 1.4679 - val_loss: 1.2130
Epoch 2/50
938/938 [==============================] - 13s 14ms/step - loss: 1.1100 - val_loss: 1.0555
Epoch 3/50
938/938 [==============================] - 13s 14ms/step - loss: 0.9841 - val_loss: 0.9726
Epoch 4/50
938/938 [==============================] - 13s 14ms/step - loss: 0.8961 - val_loss: 0.9091
Epoch 5/50
938/938 [==============================] - 13s 14ms/step - loss: 0.8328 - val_loss: 0.8717
Epoch 6/50
938/938 [==============================] - 13s 14ms/step - loss: 0.7849 - val_loss: 0.8484
Epoch 7/50
938/938 [==============================] - 13s 14ms/step - loss: 0.7467 - val_loss: 0.8289
Epoch 8/50
938/938 [==============================] - 13s 14ms/step - loss: 0.7137 - val_loss: 0.8147
Epoch 9/50
938/938 [==============================] - 13s 14ms/step - loss: 0.6894 - val_loss: 0.8115
Epoch 10/50
938/938 [==============================] - 13s 14ms/step - loss: 0.677

# Test

In [22]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         1196032   
_________________________________________________________________
masking (Masking)            (None, None, 256)         0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 525312    
Total params: 1,721,344
Trainable params: 1,721,344
Non-trainable params: 0
_________________________________________________________________


In [23]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, 256)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [24]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    1908480     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
____________________________________________________________________________________________

In [25]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [26]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [27]:
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [28]:
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [29]:
for seq_index in [1,100,301,777,2222]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: try it again . 
정답 문장: essaye encore . 
번역기가 번역한 문장:  essaie de . nouvea
-----------------------------------
입력 문장: my new car is red . 
정답 문장: ma nouvelle voiture est rouge . 
번역기가 번역한 문장:  ma ma ma la la l
-----------------------------------
입력 문장: tom became violent . 
정답 문장: tom est devenu violent . 
번역기가 번역한 문장:  tom tom tom tom 
-----------------------------------
입력 문장: are you from here ? 
정답 문장: vous tes du coin ? 
번역기가 번역한 문장:  tu d vous tu ici ic
-----------------------------------
입력 문장: i m a fast learner . 
정답 문장: j apprends vite . 
번역기가 번역한 문장:  je je e e e . . 
